In [37]:
bairro_base = input('Qual bairro de Beagá deseja consultar? ')

Qual bairro de Beagá deseja consultar? Carlos Prates


In [38]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [39]:
def scrap(url):
    c=requests.get(url).content
    soup=BeautifulSoup(c)
    aluguel_tags = soup.find_all('span', {'class':'price-tag-fraction'})
    alugueis_lista = [p.text for p in aluguel_tags]
    enderecos_tags = soup.find_all('span', {'class':'ui-search-item__group__element'})
    enderecos_lista=[end.text for end in enderecos_tags]
    enderecos_lista=enderecos_lista[1::2]
    area_quartos_tags = soup.find_all('ul', {'class':'ui-search-card-attributes'})
    area_quartos_lista=[aq.text for aq in area_quartos_tags]
    df=convert_df(bairro, area_quartos_lista, enderecos_lista, alugueis_lista)
    return df

def convert_df(bairro, area_quartos_lista, enderecos_lista, alugueis_lista):
    bairros_lista = [bairro]*len(alugueis_lista)
    data = {'bairro': bairros_lista,
            'area_quartos': area_quartos_lista,
            'enderecos_lista':enderecos_lista,
            'aluguel': alugueis_lista}
    df = pd.DataFrame(data)
    return df

In [40]:
bairro=bairro_base.lower().replace(" ","-")
URL='https://imoveis.mercadolivre.com.br/aluguel/minas-gerais/belo-horizonte/{}/{}'
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', 
           '_Desde_241', '_Desde_289','_Desde_337']
lista_dfs=[]
for pg in paginas:
   url = URL.format(bairro,pg)
   print('Coletando URL:', url)
   try:
     df = scrap(url)
   except Exception as e:
     print('Erro ao coletar URL:', url)
     print(e)  
   lista_dfs.append(df)
   sleep(1)
df_final = pd.concat(lista_dfs)

Coletando URL: https://imoveis.mercadolivre.com.br/aluguel/minas-gerais/belo-horizonte/carlos-prates/
Coletando URL: https://imoveis.mercadolivre.com.br/aluguel/minas-gerais/belo-horizonte/carlos-prates/_Desde_49
Coletando URL: https://imoveis.mercadolivre.com.br/aluguel/minas-gerais/belo-horizonte/carlos-prates/_Desde_97
Coletando URL: https://imoveis.mercadolivre.com.br/aluguel/minas-gerais/belo-horizonte/carlos-prates/_Desde_145
Coletando URL: https://imoveis.mercadolivre.com.br/aluguel/minas-gerais/belo-horizonte/carlos-prates/_Desde_193
Coletando URL: https://imoveis.mercadolivre.com.br/aluguel/minas-gerais/belo-horizonte/carlos-prates/_Desde_241
Coletando URL: https://imoveis.mercadolivre.com.br/aluguel/minas-gerais/belo-horizonte/carlos-prates/_Desde_289
Coletando URL: https://imoveis.mercadolivre.com.br/aluguel/minas-gerais/belo-horizonte/carlos-prates/_Desde_337


In [41]:
df_final['aluguel']=df_final['aluguel'].str.replace(".","").astype(float)
df_final = df_final.sort_values(['aluguel'],ascending=False)

In [42]:
df_final.head()

,bairro,area_quartos,enderecos_lista,aluguel
12,carlos-prates,"3,100 m² construídos","Carlos Prates, Belo Horizonte, Minas Gerais",165000.0
14,carlos-prates,"1,800 m² construídos","Carlos Prates, Belo Horizonte, Minas Gerais",35000.0
9,carlos-prates,"1,100 m² construídos","Avenida Dom Pedro Ii, Carlos Prates, Belo Hori...",24000.0
10,carlos-prates,"1,300 m² construídos","Avenida Nossa Senhora De Fátima, Carlos Prates...",13000.0
15,carlos-prates,800 m² construídos,"Carlos Prates, Belo Horizonte, Minas Gerais",6500.0


In [43]:
df_final['aluguel'].describe()

count       133.000000
mean       2928.609023
std       14644.091743
min         400.000000
25%         900.000000
50%        1096.000000
75%        1200.000000
max      165000.000000
Name: aluguel, dtype: float64

In [51]:
print(f"Há {int(df_final['aluguel'].count())} aptos disponíveis para aluguel no bairro {bairro_base}.")
print(f"O preço do aluguel mais caro do bairro {bairro_base} é R${int(df_final['aluguel'].max())}.")
print(f"O preço do aluguel mais barato do bairro {bairro_base} é R${int(df_final['aluguel'].min())}.")
print(f"A média de aluguel do bairro {bairro_base} é R${round(df_final['aluguel'].mean(),2)}.")

Há 133 aptos disponíveis para aluguel no bairro Carlos Prates.
O preço do aluguel mais caro do bairro Carlos Prates é R$165000.
O preço do aluguel mais barato do bairro Carlos Prates é R$400.
A média de aluguel do bairro Carlos Prates é R$2928.61.


In [7]:
df_final.to_csv(f'Dados-Aluguel-BH-{bairro}.csv',index=False,encoding='latin')